In [10]:
import os
import matplotlib.pyplot as plt
import numpy as np
import sys
sys.path.append("../src")

In [11]:
preselection_configs = [file for file in os.listdir("/afs/desy.de/user/s/spatarod/QUBO-V2/configuration/preselection/") 
                        if "c_" in file]
qubo_configs = [file for file in os.listdir("/afs/desy.de/user/s/spatarod/QUBO-V2/configuration/qubo/") 
                if ".ipynb" not in file]

In [12]:
qubo_configs

['bit_flip.yaml',
 'eigensolver_7q_alt_version_2.yaml',
 'eigensolver_7q.yaml',
 'eigensolver_16q.yaml',
 'eigensolver_7q_alt_version_3.yaml',
 'eigensolver_7q_alt_version_5.yaml',
 'eigensolver_1q.yaml',
 'eigensolver_7q_alt_version_1.yaml',
 'VQE_IdealQasmSim_7q_TwoLocal_SPSA.yaml',
 'bit_flip_alt_version_3.yaml',
 'bit_flip_alt_version_1.yaml',
 'bit_flip_alt_version_4.yaml',
 'eigensolver_7q_alt_version_4.yaml',
 'bit_flip_alt_version_5.yaml',
 'eigensolver_7q_merged_cluster.yaml',
 'bit_flip_alt_version_2.yaml',
 'VQE_IdealQasmSim_7q_TwoLocal_NFT.yaml',
 'eigensolver_5q.yaml',
 'VQE_IdealQasmSim_7q_TwoLocal_COBYLA.yaml',
 'eigensolver_10q.yaml',
 'eigensolver_12q.yaml',
 'eigensolver_3q.yaml']

In [23]:
prefix = "/nfs/dust/luxe/user/spatarod/towards_paper/e-laser/phase-0/gpc/"
xi = 5.0
preselection = "-c_3"
qubo_config = "bit_flip" 
matched_definition = 3

def get_efficiency(reco, gen, num_hits_from_same_particle):
    reco = np.load(reco, allow_pickle=True)
    gen = np.load(gen, allow_pickle=True)
    fake = 0
    matched = set()
    for xplet in reco:
        matched_xplet = False
        for particle in set(xplet.particle_ids.values()):
            if list(xplet.particle_ids.values()).count(particle) >= num_hits_from_same_particle:
                matched_xplet = True
        if matched_xplet:
            matched.add(particle)
        else:
            fake += 1
    return len(matched) / len(gen), fake / len(reco)

In [24]:
efficiency = []
fake_rate = []

for folder in os.listdir(f"{prefix}{xi}/smeared/"):
    if preselection in folder:
        for qubo in os.listdir(f"{prefix}{xi}/smeared/{folder}"):
            if "_".join(qubo.split("_")[1:]) == qubo_config:
                bx = folder.split("_")[2]
                try:
                    # print(f"{prefix}{xi}/smeared/{folder}/{qubo}")
                    eff, fake = get_efficiency(f"{prefix}{xi}/smeared/{folder}/{qubo}/reco_xplet_list.npy",
                                               f"{prefix}{xi}/e0gpc_{xi}_{bx}_sl_gen_xplet_list.npy",
                                               matched_definition)
                    efficiency.append(eff)
                    fake_rate.append(fake)
                except:
                    print(f"{prefix}{xi}/smeared/{folder}/{qubo}")
                    pass
print(f"xi = {xi}")
print(f"Efficiency {np.around(np.mean(efficiency), 3)}", "+/-", np.around(np.std(efficiency), 3))
print(f"Fake rate: {np.around(np.mean(fake_rate), 3)}", "+/-", np.around(np.std(fake_rate), 3))

xi = 5.0
Efficiency 0.927 +/- 0.003
Fake rate: 0.033 +/- 0.002


In [ ]:
Configuration c_4
xi = 7.0
Efficiency 0.667 +/- 0.004
Fake rate: 0.235 +/- 0.005
xi = 5.0
Efficiency 0.93 +/- 0.003
Fake rate: 0.03 +/- 0.00

In [ ]:
Configuration c_3
xi = 7.0
Efficiency 0.658 +/- 0.004
Fake rate: 0.248 +/- 0.004
xi = 5.0
Efficiency 0.927 +/- 0.003
Fake rate: 0.033 +/- 0.002

In [ ]:
Configuration c_5
xi = 7.0
Efficiency 0.668 +/- 0.004
Fake rate: 0.16 +/- 0.00
xi = 5.0
Efficiency 0.883 +/- 0.003
Fake rate: 0.019 +/- 0.002